In [ ]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

In [ ]:
_df = pd.read_csv('datasets/Train.csv')

In [ ]:
_df.head(15)

In [ ]:
df = pd.read_csv('datasets/Train.csv')

In [ ]:
profile = ProfileReport(df, title="Profiling Report")

In [ ]:
profile.to_file("profiling_report.html")

In [ ]:
# df columns
print(df.columns)

In [ ]:
# nulls in df columns
print(df.isnull().sum())

In [ ]:
# ignore columns call_type	call_status	start_time	duration	end_time	display_location	

df = df.drop(columns=['call_type', 'call_status', 'start_time', 'duration', 'end_time', 'display_location'])

In [ ]:
df['conversions_calls'] = df['conversions_calls'].interpolate().fillna(df['conversions_calls'].interpolate().mean())

In [ ]:
# fill nulls with 0
df = df.fillna(0)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
y = df['clicks']
X = df.drop(columns=['ID', 'clicks'])

In [ ]:
X['date'] = pd.to_datetime(X['date'])
X['year'] = X['date'].dt.year
X['month'] = X['date'].dt.month
X['day'] = X['date'].dt.day
X = X.drop(['date'], axis=1)

In [ ]:
X['currency'] = X['currency'].astype(str)
X['ad_type'] = X['ad_type'].astype(str)

In [ ]:
# encode currency, ad_type
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X['currency'] = le.fit_transform(X['currency'])
X['ad_type'] = le.fit_transform(X['ad_type'])


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
# evaluate model
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)

print(mse)

In [ ]:
# predict on the entire dataset

x_pred = model.predict(X)
x_pred = x_pred.astype(int)

In [ ]:
len(_df), len(X), len(x_pred)

In [ ]:
df.tail()

In [ ]:
def create_submission_id(df):
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['submission_id'] = df['ID'].astype(str) + '_' + df['year'].astype(str) + '_' + df['month'].astype(str) + '_' + df['day'].astype(str)
    return df

# Apply the function to your DataFrame
df = create_submission_id(df)

In [ ]:
submission_df = pd.DataFrame()
submission_df['ID'] = df['ID']
submission_df['date'] = df['date']
submission_df['clicks'] = x_pred

In [ ]:
submission_df.to_csv("random_forest_regressor_.csv", index=False)

In [ ]:
def generate_weekly_clicks_df(df):
    # Parse the 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Group the data by 'ID' and iterate over each group
    rows = []
    for id, group in df.groupby('ID'):
        # Get the maximum date for the ID
        max_date = group['date'].max()

        # Find the previous Monday of the week for the maximum date
        prev_monday = max_date - pd.Timedelta(days=(max_date.dayofweek + 7) % 7)

        # Calculate the last date of the first week (Sunday)
        first_week_end = prev_monday + pd.Timedelta(days=6)

        # Calculate the last date of the second week (Sunday)
        second_week_end = first_week_end + pd.Timedelta(days=7)

        # Create rows for the end dates of the first and second weeks
        first_week_row = {'ID': f"{id}_{first_week_end.year}_{first_week_end.month:02d}_{first_week_end.day:02d}", 'clicks': 0}
        second_week_row = {'ID': f"{id}_{second_week_end.year}_{second_week_end.month:02d}_{second_week_end.day:02d}", 'clicks': 0}

        # Append the rows to the list
        rows.append(first_week_row)
        rows.append(second_week_row)

    # Create a DataFrame from the list of rows
    result_df = pd.DataFrame(rows)

    return result_df

# Example usage:
# Assuming 'df' is your dataframe containing the advertising data
# weekly_clicks_df = generate_weekly_clicks_df(df)


In [ ]:

c_df = generate_weekly_clicks_df(submission_df)

In [ ]:
len(c_df)

In [ ]:
c_df.to_csv("grouped_dates.csv", index=False)